# NBER recessions

In [57]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [33]:
# load file with custon plotly functions for timeseries, 
# color scheeme and pandas read to concat series into tables
%run ../zillow/code/functions/nice_plot_pandas_getdata.ipynb

In [137]:
# get recession form FRED and used customized functions to create dataframes
start  = dt.datetime(1854, 10, 1) 
s = ['NBER Recessions [USRECQM]']
dic = dict_from_FRED_description(s)
df = make_hrizontal_tbl(dic, start=start)

# recessions are marked with 1. No recesion are marked with 0.
plot_nice(title='NBER recessions', dic=dic, df=df, 
          margin=dict(autoexpand=False,l=120,r=120,t=110,), 
          show_series_label=False,
         y_units='%')

In [40]:
#look at df
df.head()

,USRECQM
DATE,
1854-10-01,1
1855-01-01,0
1855-04-01,0
1855-07-01,0
1855-10-01,0
...,...
2019-01-01,0
2019-04-01,0
2019-07-01,0


In [88]:
# get initial and final dates (where markers change from 0->1 and 1->0)
init_date = df[(df.USRECQM == 1) & (df.USRECQM.shift() != 1)].index.to_list()
end_date =  df[(df.USRECQM == 1) & (df.USRECQM.shift(-1) != 1)].index.to_list()

In [89]:

fig = go.Figure()

# Add scatter trace for line
fig.add_trace(go.Scatter(
    x=[min(init_date), max(init_date)],
    y=[0,0],
    mode="markers",
    marker=dict(size=1), ))


fig.update_layout(
        xaxis=dict(showline=True,showgrid=True,showticklabels=True,
                   linecolor='rgb(204, 204, 204)',linewidth=2,ticks='outside',
                   tickfont=dict(family='Arial',size=12,color='rgb(82, 82, 82)',),),
        yaxis=dict(showgrid=True,zeroline=False,showline=True,showticklabels=True,gridcolor='rgb(204, 204, 204)'),
        
        autosize=False,
        showlegend=False,
        plot_bgcolor='white'
    )

# Add shape regions

shapes = []
for init_date, end_date in list(zip(init_date, end_date)):
    shapes.append(dict(
                        type="rect",
                        # x-reference is assigned to the x-values
                        xref="x",
                        # y-reference is assigned to the plot paper [0,1]
                        yref="paper",
                        x0=init_date,
                        y0=0,
                        x1=end_date,
                        y1=1,
                        fillcolor="lightgray",
                        opacity=0.5,
                        layer="below",
                        line_width=0,
                    ),)
    

fig.update_layout(shapes=shapes)


fig.show()

In [ ]:
# get recession form FRED and used customized functions to create dataframes
start  = dt.datetime(1854, 10, 1) 
s = ['NBER Recessions [USRECQM]']
dic = dict_from_FRED_description(s)
df = make_hrizontal_tbl(dic, start=start)

In [15]:
df.head(10)

,USRECQM
DATE,
1960-01-01,0
1960-04-01,1
1960-07-01,1
1960-10-01,1
1961-01-01,1
1961-04-01,0
1961-07-01,0
1961-10-01,0
1962-01-01,0


In [135]:
s = ['Homeownership Rate for the United States [RHORUSQ156N]']
dic = dict_from_FRED_description(s)
df = make_hrizontal_tbl(dic, start=start)
plot_nice(title='Homeownership Rate', dic=dic, df=df, 
          margin=dict(autoexpand=False,l=120,r=120,t=110,), 
          show_series_label=False,
          y_units='%')

In [136]:
def plot_nice(title='', dic=dic, df=df, margin=dict(autoexpand=False,l=100,r=120,t=110,), 
              vertical_label_gutter=60, show_series_label=True, log = False, 
              show_endpoints=True, 
              source = 'U.S. Census Bureau', height=450, y_units='', 
              colors = col_scheeme, line_wid=1.5, recessions=True):
    
    labels = [values for values in dic.values()] #['Occupied Housing Units ','Owner Occupied Housing Units ','Renter Occupied Housing Units ']

    # color coding
    #colors = px.colors.sequential.haline

    dic_colors=pd.Series(colors[:len(dic)])
    dic_colors.index=[key for key in dic.keys()] 
    dic_colors = dict(dic_colors)

    # if x axes do are different, make a stack for looping on it
    x_data = df.index

    fig = go.Figure()

    for key, value in dic.items():
        fig.add_trace(go.Scatter(x=x_data, y=df[key], mode='lines',
            name=value,
            line=dict(color=dic_colors[key], width=line_wid),
            connectgaps=True,
        ))

        # endpoints
        fig.add_trace(go.Scatter(
            x=[x_data[0], x_data[-1]],
            #x=[x_data[i][0], x_data[i][-1]], # if x values are different
            y=[list(df[key])[0], list(df[key])[-1]],
            mode='markers',
            marker=dict(color=dic_colors[key], size=1)
        ))

    fig.update_layout(
        xaxis=dict(showline=True,showgrid=True,showticklabels=True,
                   linecolor='rgb(204, 204, 204)',linewidth=2,ticks='outside',
                   tickfont=dict(family='Arial',size=12,color='rgb(82, 82, 82)',),),
        yaxis=dict(showgrid=True,zeroline=False,showline=True,showticklabels=True,gridcolor='rgb(204, 204, 204)'),
        
        autosize=False,
        margin=margin,
        showlegend=False,
        plot_bgcolor='white'
    )

    annotations = []

    # Adding labels
    for key in dic.keys():  

        # label each line
        
        if show_series_label:
          
        ################################### manually adjusting label position #############################    
            if ((key != 'NYXRCNSA')& (key != 'SFXRCNSA') &
                (key != 'SFXRNSA')& (key != 'DAXRNSA') & 
                (key != 'SEXRNSA') & (key != 'SFXRNSA')&
                (key != 'LXXRHTNSA') & (key != 'NYXRHTNSA')&
                (key != 'LVXRLTNSA') & (key != 'SFXRLTNSA')&(key != 'SEXRLTNSA')
               ):
                
                annotations.append(dict(xref='paper', x=0.95, y=df[key][-1]+vertical_label_gutter,
                                              xanchor='left', yanchor='middle',
                                              #text=dic[key] + ' {}%'.format(df[key][0]),
                                              text=dic[key] ,
                                              font=dict(family='Arial',size=15, color=dic_colors[key]),
                                              showarrow=False))
            elif ((key == 'NYXRCNSA') | (key == 'DAXRNSA')):
                annotations.append(dict(xref='paper', x=0.95, y=df[key][-1]+vertical_label_gutter-5,
                                              xanchor='left', yanchor='middle',
                                              #text=dic[key] + ' {}%'.format(df[key][0]),
                                              text=dic[key] ,
                                              font=dict(family='Arial',size=15, color=dic_colors[key]),
                                              showarrow=False))
            elif ((key == 'SFXRCNSA')|(key == 'SFXRNSA')|(key == 'LVXRLTNSA')|(key == 'SFXRLTNSA')|(key == 'SEXRLTNSA')):
                annotations.append(dict(xref='paper', x=0.95, y=df[key][-1]+vertical_label_gutter+5,
                                              xanchor='left', yanchor='middle',
                                              #text=dic[key] + ' {}%'.format(df[key][0]),
                                              text=dic[key] ,
                                              font=dict(family='Arial',size=15, color=dic_colors[key]),
                                              showarrow=False))
            elif (key == 'SEXRNSA'):
                annotations.append(dict(xref='paper', x=0.95, y=df[key][-1]+vertical_label_gutter-3.5,
                                              xanchor='left', yanchor='middle',
                                              #text=dic[key] + ' {}%'.format(df[key][0]),
                                              text=dic[key] ,
                                              font=dict(family='Arial',size=15, color=dic_colors[key]),
                                              showarrow=False))
                
            elif ((key == 'LXXRHTNSA') | (key == 'NYXRHTNSA')):
                annotations.append(dict(xref='paper', x=0.95, y=df[key][-1]+vertical_label_gutter+7,
                                              xanchor='left', yanchor='middle',
                                              #text=dic[key] + ' {}%'.format(df[key][0]),
                                              text=dic[key] ,
                                              font=dict(family='Arial',size=15, color=dic_colors[key]),
                                              showarrow=False))


        ################################### manually adjusting label position #############################

        
        # making k if number is too large
        text = ('{:.0f}'.format(list(df[key])[0]) if (list(df[key])[0] <1000) else '{:.0f}k'.format(list(df[key])[0]/1000))
        text = (text if show_endpoints else '')
        
        # labeling the left_side of the plot
        annotations.append(dict(xref='paper', x=0.05, y=list(df[key])[0],
                                      #xanchor='right', yanchor='middle',
                                      xanchor='right', yanchor='top',
                                      #text=dic[key] + ' {}%'.format(df[key][0]),
                                      #text=dic[key] + '{:.0f}k'.format(df[key][0]/1000), # with line label
                                      text=text,
                                      font=dict(family='Arial',size=12),
                                      showarrow=False,
                                      ))
        
        
        # making k if number is too large
        text = ('{:.0f}'.format(list(df[key])[-1]) if (list(df[key])[-1] <1000) else '{:.0f}k'.format(list(df[key])[-1]/1000))
        text = (text if show_endpoints else '')
    
        # labeling the right_side of the plot
        annotations.append(dict(xref='paper', x=0.95, y=list(df[key])[-1],
                                      xanchor='left', yanchor='top',
                                      #text='{}%'.format(df[key][-1]),
                                      text= text,
                                      font=dict(family='Arial',size=12),
                                      showarrow=False))
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                                      xanchor='left', yanchor='bottom',
                                      text=title,
                                      font=dict(family='Arial',size=22,color='rgb(37,37,37)'),
                                      showarrow=False))
    # Source
    annotations.append(dict(xref='paper', yref='paper', x=0, y=-0.1,
                                  xanchor='left', yanchor='top',
                                  text='Source: ' + source,
                                  font=dict(family='Arial',size=12, color='rgb(150,150,150)'),
                                  showarrow=False))
    
    #units on y-axis
    annotations.append(dict(xref="paper",yref="paper", x= 0.0,y=1.03,#textangle=-90,
                            font=dict(family='Arial',size=12, color='black'),
                            showarrow=False,text = y_units,))

    fig.update_layout(annotations=annotations, height=height)

    if log:
        fig.update_layout(yaxis=dict(type= 'log'))
    
    # add recessions
    if recessions:
        # get recession form FRED and used customized functions to create dataframes
        s = ['NBER Recessions [USRECQM]']
        dic = dict_from_FRED_description(s)
        df = make_hrizontal_tbl(dic, start=min(df.index))
        
        # get initial and final dates (where markers change from 0->1 and 1->0)
        init_date = df[(df.USRECQM == 1) & (df.USRECQM.shift() != 1)].index.to_list()
        end_date =  df[(df.USRECQM == 1) & (df.USRECQM.shift(-1) != 1)].index.to_list()
        # Add shape regions

        shapes = []
        i = 0
        for init_date, end_date in list(zip(init_date, end_date)):
            shapes.append(dict(
                                type="rect",
                                # x-reference is assigned to the x-values
                                xref="x",
                                # y-reference is assigned to the plot paper [0,1]
                                yref="paper",
                                x0=init_date,
                                y0=0,
                                x1=end_date,
                                y1=1,
                                fillcolor="lightgray",
                                line=dict(color="white",width=.1,),
                                #opacity=0.5,
                                layer="below",
                                line_width=0,
                                
                            ),)
            i+=1 # count how many recessions to use on "visible" multiplier
    

        #fig.update_layout(shapes=shapes)
        
        fig.update_layout(
        updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=1,
            y=1.2,
            buttons=list([
                dict(label="-",
                     method="relayout",
                     args=["shapes", []]),
                
                dict(label="+",
                     method="relayout",
                     args=["shapes", shapes])
                
                        ]),
            )
        ]) # end of update layout
        
    fig.show()